In [3]:
import pandas as pd
import geopandas as gpd
import rasterio
import glob
import matplotlib.pyplot as plt
import numpy as np

In [17]:
home = '/global/scratch/users/erin_carroll/SUFHER/validation_data/forest_health/'
tif_folder = '/global/scratch/users/erin_carroll/data/tifs/GMUG_/'

In [48]:
plots = pd.read_csv(home+'forest_health_plots.csv')
plots = plots[plots.Year==2013] # use 2013 GPS locations
plots = plots.drop(columns=['Year', 'date']) # timeless, time to come from raster values
plots = gpd.GeoDataFrame(plots, geometry=gpd.points_from_xy(plots['GPS.E'], plots['GPS.N']), crs='EPSG:32613')
plots_12N = plots.to_crs(epsg=32612)

In [54]:
bands = ['B02','B03','B04','B05','B06','B07','B08','B8A','B11','B12']

In [43]:
tif_fps = [x for x in sorted(glob.glob(tif_folder+'/*/*/*/*_mosaic.tif')) if '0m' not in x]
len(tif_fps)

240

In [71]:
gdfs = []
for tif_fp in tif_fps:
    name = tif_fp.split('/')[-1]
    print(name)
    t = name.split('_')[0]
    y = name.split('_')[1]
    m = name.split('_')[2]

    with rasterio.open(tif_fp) as src:
        if src.crs=='EPSG:32612':
            plots_ = plots_12N.copy()
        else:
            plots_ = plots.copy()
        coords = [(point.x, point.y) for point in plots_.geometry]
        plots_['tmp'] = list(src.sample(coords))
        for b in range(len(bands)):
            plots_[f'{bands[b]}'] = plots_['tmp'].apply(lambda x: x[b] if x is not None else None)
        plots_.drop(columns=['tmp'], inplace=True)
    plots_['tile'] = t
    plots_['year'] = y
    plots_['month'] = m
    gdfs.append(plots_)

12SXH_2017_aug_mosaic.tif
12SXH_2017_jul_mosaic.tif
12SXH_2017_sep_mosaic.tif
12SXH_2018_aug_mosaic.tif
12SXH_2018_jul_mosaic.tif
12SXH_2018_sep_mosaic.tif
12SXH_2019_aug_mosaic.tif
12SXH_2019_jul_mosaic.tif
12SXH_2019_sep_mosaic.tif
12SXH_2020_aug_mosaic.tif
12SXH_2020_jul_mosaic.tif
12SXH_2020_sep_mosaic.tif
12SXH_2021_aug_mosaic.tif
12SXH_2021_jul_mosaic.tif
12SXH_2021_sep_mosaic.tif
12SXH_2022_aug_mosaic.tif
12SXH_2022_jul_mosaic.tif
12SXH_2022_sep_mosaic.tif
12SXH_2023_aug_mosaic.tif
12SXH_2023_jul_mosaic.tif
12SXH_2023_sep_mosaic.tif
12SXH_2024_aug_mosaic.tif
12SXH_2024_jul_mosaic.tif
12SXH_2024_sep_mosaic.tif
12SXJ_2017_aug_mosaic.tif
12SXJ_2017_jul_mosaic.tif
12SXJ_2017_sep_mosaic.tif
12SXJ_2018_aug_mosaic.tif
12SXJ_2018_jul_mosaic.tif
12SXJ_2018_sep_mosaic.tif
12SXJ_2019_aug_mosaic.tif
12SXJ_2019_jul_mosaic.tif
12SXJ_2019_sep_mosaic.tif
12SXJ_2020_aug_mosaic.tif
12SXJ_2020_jul_mosaic.tif
12SXJ_2020_sep_mosaic.tif
12SXJ_2021_aug_mosaic.tif
12SXJ_2021_jul_mosaic.tif
12SXJ_2021_s

In [74]:
plots_spectra = pd.concat(gdfs, ignore_index=True)
plots_spectra


Plot  plot_number plot_type         GPS.N          GPS.E GPS.Zone  \
0      002S            2         S  4.327647e+06  232676.182438      13N   
1      002H            2         H  4.327428e+06  232759.925395      13N   
2      004S            4         S  4.332635e+06  248446.000000      13N   
3      004H            4         H  4.332416e+06  248424.000000      13N   
4      005S            5         S  4.348016e+06  272304.000000      13N   
...     ...          ...       ...           ...            ...      ...   
38875  311S          311         S  4.160074e+06  213149.100248      13N   
38876  312H          312         H  4.163346e+06  206992.292417      13N   
38877  312S          312         S  4.163019e+06  207490.646123      13N   
38878  316H          316         H  4.176861e+06  216116.582382      13N   
38879  316S          316         S  4.176557e+06  216118.140504      13N   

       FLAG_plot_location_discrepancy                        geometry  B02  \
0                                   0  POINT (751859.000 4327136.000)    0   
1                                   0  POINT (751957.000 4326923.000)    0   
2                                   1  POINT (767265.173 4333154.368)    0   
3                                   0  POINT (767257.683 4332934.371)    0   
4                                   0  POINT (790059.214 4350083.999)    0   
...                               ...                             ...  ...   
38875                               0  POINT (213149.100 4160074.451)    0   
38876                               0  POINT (206992.292 4163346.023)    0   
38877                               0  POINT (207490.646 4163019.433)    0   
38878                               0  POINT (216116.582 4176860.653)    0   
38879                               0  POINT (216118.141 4176556.853)    0   

       B03  ...  B05  B06  B07  B08  B8A  B11  B12   tile  year month  
0        0  ...    0    0    0    0    0    0    0  12SXH  2017   aug  
1        0  ...    0    0    0    0    0    0    0  12SXH  2017   aug  
2        0  ...    0    0    0    0    0    0    0  12SXH  2017   aug  
3        0  ...    0    0    0    0    0    0    0  12SXH  2017   aug  
4        0  ...    0    0    0    0    0    0    0  12SXH  2017   aug  
...    ...  ...  ...  ...  ...  ...  ...  ...  ...    ...   ...   ...  
38875    0  ...    0    0    0    0    0    0    0  13SCD  2024   sep  
38876    0  ...    0    0    0    0    0    0    0  13SCD  2024   sep  
38877    0  ...    0    0    0    0    0    0    0  13SCD  2024   sep  
38878    0  ...    0    0    0    0    0    0    0  13SCD  2024   sep  
38879    0  ...    0    0    0    0    0    0    0  13SCD  2024   sep  

[38880 rows x 21 columns]

In [80]:
plots_spectra[bands] = plots_spectra[bands].replace(0, np.nan)

In [83]:
plots_spectra['NDVI'] = (plots_spectra['B08'] - plots_spectra['B04'])/(plots_spectra['B08'] + plots_spectra['B04'])
plots_spectra['NDMI'] = (plots_spectra['B08'] - plots_spectra['B11'])/(plots_spectra['B08'] + plots_spectra['B11'])

In [98]:
# for each ponit, reduce to the values from the tile with the max NDVI

plots_spectra_summary = plots_spectra.loc[
    plots_spectra.groupby(['Plot', 'year', 'month'])['NDVI'].idxmax(), # get row index with max NDVI
    ['Plot', 'year', 'month', 'tile', *bands, 'NDVI', 'NDMI']
].reset_index(drop=True)

In [105]:
plots_spectra_summary.columns

Index(['Plot', 'year', 'month', 'tile', 'B02', 'B03', 'B04', 'B05', 'B06',
       'B07', 'B08', 'B8A', 'B11', 'B12', 'NDVI', 'NDMI'],
      dtype='object')

In [106]:
# export
fp_out = home+'plots_spectra.csv'
plots_spectra_summary.to_csv(fp_out, index=False)